In [7]:
import torch
import dxtb
from dxtb.typing import DD

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}

numbers = torch.tensor([6, 6, 7, 7, 1, 1, 1, 1, 1, 1, 8, 8,], device=dd["device"])
positions = torch.tensor([
                [-3.81469488143921, +0.09993441402912, 0.00000000000000],
                [+3.81469488143921, -0.09993441402912, 0.00000000000000],
                [-2.66030049324036, -2.15898251533508, 0.00000000000000],
                [+2.66030049324036, +2.15898251533508, 0.00000000000000],
                [-0.73178529739380, -2.28237795829773, 0.00000000000000],
                [-5.89039325714111, -0.02589114569128, 0.00000000000000],
                [-3.71254944801331, -3.73605775833130, 0.00000000000000],
                [+3.71254944801331, +3.73605775833130, 0.00000000000000],
                [+0.73178529739380, +2.28237795829773, 0.00000000000000],
                [+5.89039325714111, +0.02589114569128, 0.00000000000000],
                [-2.74426102638245, +2.16115570068359, 0.00000000000000],
                [+2.74426102638245, -2.16115570068359, 0.00000000000000],
                ], **dd) # ** to use dd as kwargs

calc = dxtb.calculators.GFN1Calculator(numbers,  **dd) # Better for argument checking
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd)

# compute the energy
pos = positions.clone().requires_grad_(True)
energy = calc.get_energy(pos, verbosity = 0)

Total Energy: -23.28352325126154 Hartree.


In [2]:
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd)
energy = calc.get_energy(positions, spin=2)

Total Energy: -23.08744880283994 Hartree.


# Check shape of P, E and C if I change the spin

In [3]:
from dxtb.config import ConfigCache

cache_config = ConfigCache(density=True, coefficients=True, mo_energies=True)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd)
calc.opts.cache = cache_config

density_spinNone = calc.get_density(pos, spin=None)
density_spin0 = calc.get_density(pos, spin=6)
# coefficients = calc.get_coefficients(pos)
# energies = calc.get_mo_energies(pos)

print(f"density_spinNone.shape = {density_spinNone.shape}")
print(f"density_spin0.shape = {density_spin0.shape}")


Total Energy: -23.28352325126154 Hartree.
Total Energy: -22.51576773181534 Hartree.


density_spinNone.shape = torch.Size([36, 36])
density_spin0.shape = torch.Size([36, 36])


# Check from tblite

In [4]:
from tblite.interface import Calculator

calc_tblite = Calculator(
    method = "GFN1-xTB",
    numbers = numbers.numpy(),
    positions = positions.numpy(),
    uhf=0
)

calc_tblite.add("spin-polarization", 1.0)
calc_tblite.set("save-integrals", 1)
res = calc_tblite.singlepoint()

P = res["density-matrix"]

print(f"P.shape = {P.shape}")

------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -22.85385429255  -2.3068396E+01   4.7115501E-01
      2     -23.27567606155  -4.2182177E-01   2.3065474E-01
      3     -23.28258085318  -6.9047916E-03   1.1021572E-01
      4     -23.28330667123  -7.2581805E-04   1.4057292E-02
      5     -23.28349930909  -1.9263785E-04   5.4079723E-03
      6     -23.28352212644  -2.2817352E-05   8.2885093E-04
      7     -23.28352167143   4.5500649E-07   6.9884287E-04
      8     -23.28352428611  -2.6146786E-06   1.5998298E-05
      9     -23.28352428643  -3.1952396E-10   1.5132300E-05
------------------------------------------------------------

 total:                                   0.012 sec
P.shape = (2, 36, 36)


# Check interaction term dxtb

In [8]:
from dxtb.components.base import Interaction

inter = Interaction(**dd)
